# Clasificación de textos utilizando algoritmos de Regresión Logística y Random Forest (sklearn)

- El presente tutorial se inspira de: https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

En este tutorial intentaremos clasificar reseñas de productos Amazon (Alexa) en dos categorías: positivos o negativos ("Análisis de sentimientos").

Utilizaremos un enfoque simple:
- representaremos los textos con representaciones vectoriales "Bag of Words"
- utilizaremos algoritmos de Machine Learning para aprender modelos a partir de las representaciones vectoriales
- evaluaremos los modelos utilizando una matriz de confusión

In [1]:
!python -m spacy download en_core_web_sm

2022-10-25 18:07:56.524374: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 18:08:00.159043: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-25 18:08:00.159279: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-25 18:08:00.525518: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-25 18:08:11.627659: W tensorflow/stream_executor/platform/de

In [2]:
#NLP
import spacy
nlp = spacy.load("en_core_web_sm")
print(spacy.__version__)
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string

#SKLEARN
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression # Regresion Logística

#PANDAS
import pandas as pd

2022-10-25 18:10:08.340345: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 18:10:08.542618: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-25 18:10:08.542639: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-25 18:10:08.577228: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-25 18:10:09.336488: W tensorflow/stream_executor/platform/de

3.2.4


# 2. Dataset: Alexa

Vamos a usar un conjunto de datos real: un conjunto de reseñas de productos de Amazon Alexa. Este conjunto de datos viene como un archivo separado por tabulaciones (.tsv). Tiene cinco columnas: 
- __rating__: se refiere a la calificación que cada usuario dio a Alexa (de 0 a 5). 
- __fecha__: fecha de la reseña
- __variación__: describe el modelo de producto Alexa que el usuario comentó.
- __verified_reviews__: contiene el texto del comentario.
- __feedback__: contiene un label, 0 o 1, que indica el sentimiento general negativo (0) o positivo (1).

In [3]:
# Loading TSV file
df_amazon = pd.read_csv("data/amazon_alexa.tsv", sep="\t")

In [4]:
# Top 5 records
df_amazon

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [5]:
# shape of dataframe
df_amazon.shape

(3150, 5)

In [6]:
# Feedback Value count
df_amazon.feedback.value_counts()

1    2893
0     257
Name: feedback, dtype: int64

#### Partición de los datos en conjuntos de entrenamiento y test para entrenar y evaluar un modelo predictivo

Usaremos la mitad de nuestro conjunto de datos como nuestro conjunto de entrenamiento, que incluirá las respuestas correctas. Luego probaremos nuestro modelo usando la otra mitad del conjunto de datos sin darle las respuestas, para ver con qué precisión funciona.

Convenientemente, scikit-learn nos da una función incorporada para hacer esto: train_test_split(). Sólo necesitamos decirle el conjunto de características que queremos que se divida (X), las etiquetas contra las que queremos que se realice la prueba (ylabels), y el tamaño que queremos usar para el conjunto de pruebas (representado como un porcentaje en forma decimal).

In [7]:
X = df_amazon['verified_reviews'] # the features we want to analyze
ylabels = df_amazon['feedback'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.5)

In [8]:
X

0                                           Love my Echo!
1                                               Loved it!
2       Sometimes while playing a game, you can answer...
3       I have had a lot of fun with this thing. My 4 ...
4                                                   Music
                              ...                        
3145    Perfect for kids, adults and everyone in betwe...
3146    Listening to music, searching locations, check...
3147    I do love these things, i have them running my...
3148    Only complaint I have is that the sound qualit...
3149                                                 Good
Name: verified_reviews, Length: 3150, dtype: object

# 3. Preprocesamientos y representación vectorial

Crearemos una función personalizada <code>spacy_tokenizer()</code> que acepta una frase como entrada y la procesa en tokens, realizando lemmatización, minúsculas y eliminando palabras stop-words.

In [9]:
# Create our list of punctuation marks
punctuations = [".",",","!","?", "#","&"]

# Create our list of stopwords
stop_words=[""]

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lower_ for word in mytokens]
        
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

#### Vectorización de los textos en BoW o TF-IDF, con scikit-learn

Podemos generar una matriz BoW para nuestros datos de texto usando la clase <code>CountVectorizer</code> de scikit-learn. En el código de abajo, le decimos a CountVectorizer que use la función personalizada spacy_tokenizer que construimos como su tokenizer, y que defina el rango de ngramo que queremos.

Los N-gramos son combinaciones de palabras adyacentes en un texto dado, donde _n_ es el número de palabras que se incluyen en las fichas. Por ejemplo, en la frase "¿Quién ganará la Copa del Mundo de fútbol en 2022? Bigramas sería una secuencia de dos palabras contiguas como "quién ganará", "ganará la", y así sucesivamente. Así que el parámetro ngram_range que usaremos en el código de abajo establece los límites inferior y superior de nuestros ngramas (usaremos unigramas). Entonces asignaremos los ngramas a bow_vector.

In [10]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

Podemos también transformar los textos en vectores para tener los pesos TF-IDF de cada palabra en cada documento:

In [11]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

# 4. Entrenamiento del modelo de clasificación

Es el momento de construir nuestro modelo predictivo. Empezaremos importando el módulo LogisticRegression y creando un objeto clasificador LogisticRegression.

Luego, crearemos un pipeline de procesamiento con dos componentes: un vectorizador y algoritmo de clasificación basado en la regresión logística. El vectorizador utiliza preprocesamientos (spacy) y vectorización (scikit-learn) para crear una matriz para representar nuestros textos.

Una vez que se construya este pipeline, se aprende el modelo predictivo llamando el método fit().

- Regresión Logística sobre representacion vectorial "BoW"

In [12]:
# Logistic Regression Classifier
modelLR = LogisticRegression()

# Create pipeline using Bag of Words
model1 = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelLR)])

# model generation
model1.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f75daf819d0>)),
                ('regression-ML', LogisticRegression())])

- Regresión Logística sobre representacion vectorial "TF-IDF"

In [13]:
# Logistic Regression Classifier
modelLR = LogisticRegression()

model2 = Pipeline([('preprocessing', tfidf_vector),
                 ('regression-ML', modelLR)])

# model generation
model2.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x7f75daf819d0>)),
                ('regression-ML', LogisticRegression())])

- Random Forest sobre representacion vectorial "BoW"

In [14]:
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=0)

model3 = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelRF)])

# model generation
model3.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f75daf819d0>)),
                ('regression-ML', RandomForestClassifier(random_state=0))])

# 5. Evaluación del modelo de clasificación

In [27]:
# Predicting with a test dataset
predicted = model1.predict(X_test)
print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

[1 1 1 ... 1 1 1]
Logistic Regression Accuracy: 0.9365079365079365
Logistic Regression Precision: 0.94067237969677
Logistic Regression Recall: 0.9930410577592206


In [28]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predicted)
print(confusion_matrix)

#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

[[  48   90]
 [  10 1427]]
              precision    recall  f1-score   support

           0       0.83      0.35      0.49       138
           1       0.94      0.99      0.97      1437

    accuracy                           0.94      1575
   macro avg       0.88      0.67      0.73      1575
weighted avg       0.93      0.94      0.92      1575



In [29]:
def printNMostInformative(vectorizer, model, N):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(model.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)

In [30]:
printNMostInformative(bow_vector, modelLR, 20)

Class 1 best: 
(-3.041985892838304, 'not')
(-1.4639288256531542, 'back')
(-1.438688201121309, 'terrible')
(-1.4171451779980204, 'working')
(-1.3169642931766559, ' ')
(-1.2651698372293538, 'does')
(-1.1789509787712433, 'n’t')
(-1.0936740127102857, 'after')
(-1.0771754470196913, "n't")
(-1.0495686882016453, 'stopped')
(-1.0268504071732543, 'poor')
(-1.003691422925835, 'work')
(-0.9464029924581475, 'the')
(-0.9369719908618762, 'hardly')
(-0.9014437384914352, 'this')
(-0.8837361065888238, 'will')
(-0.878431219031461, 'low')
(-0.8777103527353189, 'features')
(-0.8762604661504177, 'no')
(-0.8585147323207976, '..')
Class 2 best: 
(2.511588006960702, 'love')
(1.813440413650868, 'great')
(1.2990465578168795, 'easy')
(1.1372186873977983, 'like')
(0.8668221351635402, 'my')
(0.835829190559999, 'works')
(0.7995099451002685, 'alexa')
(0.7533529210212707, 'new')
(0.7430799895332303, 'for')
(0.7413385258455675, 'she')
(0.7187801210504172, 'nice')
(0.6930040896412816, 'with')
(0.6633406673077986, 'lear

/home/mvernier/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
